In [1]:
import torch 
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor, Compose, Resize, TrivialAugmentWide, Normalize
from torch.nn import CrossEntropyLoss, Softmax
import torchvision.models as models
import glob
import os
import PIL 
import ipyplot
import tqdm
import cv2
import numpy as np
import random
import opendatasets as od
from dataclasses import dataclass

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
od.download("https://www.kaggle.com/competitions/plant-seedlings-classification/data")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading plant-seedlings-classification.zip to .\plant-seedlings-classification


100%|██████████| 1.69G/1.69G [02:28<00:00, 12.2MB/s]



Extracting archive .\plant-seedlings-classification/plant-seedlings-classification.zip to .\plant-seedlings-classification


In [4]:
image_url = './plant-seedlings-classification/train/*/*.*'
image_list = []
category = []
import matplotlib.pyplot as plt
all_files =  glob.glob(image_url)
for i in all_files:
    image_list.append(i)
    category.append(i.split('\\')[-2])
    

In [5]:
ipyplot.plot_class_tabs( image_list, category,max_imgs_per_tab=26, img_width=150)

In [6]:
def mean_std_images(image_url:str, sample:int) -> tuple:
    means = np.array([0, 0, 0], dtype=np.float32)
    stds = np.array([0, 0, 0], dtype=np.float32)
    total_images = 0
    randomly_sample = sample
    for f in tqdm.tqdm(random.sample(glob.glob(image_url, recursive = True), randomly_sample)):
        img = cv2.imread(f)
        means += img.mean(axis=(0,1))
        stds += img.std(axis=(0,1))
        total_images += 1
    means = means / (total_images * 255.)
    stds = stds / (total_images * 255.)
    return means, stds


In [7]:
mean_std_images('./plant-seedlings-classification/train/*/*.*', 3000)

100%|██████████| 3000/3000 [01:16<00:00, 39.27it/s]


(array([0.20678197, 0.28868127, 0.3280315 ], dtype=float32),
 array([0.10643905, 0.09730922, 0.093356  ], dtype=float32))

In [8]:
all_files =  glob.glob('./plant-seedlings-classification/train/*/*.*')
torch.randint(int(len(all_files)*0.4), int(len(all_files)*0.5), (1,))

tensor([1924])

In [9]:
@dataclass
class PreprocessConfiguration:
    batch_size: int = 32
    resize:int = 64
    train_size: float = 0.8
    image_url_for_std: str = './plant-seedlings-classification/train/*/*.*'
    image_url_for_train: str = './plant-seedlings-classification/train/'
    num_workers:int = os.cpu_count()
       

In [27]:

def preprocess_image_folder_data( preprocessing_configuration = PreprocessConfiguration()):
    print('Step 1: Preprocessing Image')
    all_files =  glob.glob(preprocessing_configuration.image_url_for_train)

    print('Step 1.1: Randomly calculating mean and standard for Train Transform normalize')

    mean, std = mean_std_images(preprocessing_configuration.image_url_for_std, 3000) #torch.randint(int(len(all_files)*0.4), int(len(all_files)*0.5), (1,)).item())

    train_transform =  Compose([Resize((preprocessing_configuration.resize,preprocessing_configuration.resize)), 
                        TrivialAugmentWide(num_magnitude_bins=31), 
                        ToTensor()
                        #Normalize(mean=mean,std=std) 
                                    ]
                        )
    print('Step 1.2: Loading Image from folders')
    
    full_train_dataset = ImageFolder(
        root=preprocessing_configuration.image_url_for_train,
        transform= train_transform
            )
    train_size = int(preprocessing_configuration.train_size * len(full_train_dataset))
    test_size = len(full_train_dataset) - train_size
    
    print('Step 1.3: Train/Test Split Datasets')
    train_data, test_data = torch.utils.data.random_split(full_train_dataset, [train_size, test_size])

    BATCH_SIZE = preprocessing_configuration.batch_size

    train_loader = DataLoader(
        train_data, batch_size=BATCH_SIZE, shuffle=True,
        num_workers=preprocessing_configuration.num_workers, pin_memory=True
    )
    
    valid_loader = DataLoader(
        test_data, batch_size=BATCH_SIZE, shuffle=False,
        num_workers=preprocessing_configuration.num_workers, pin_memory=True, 
    )
    return train_loader, valid_loader

In [11]:
(list(models.resnet101(pretrained=True).children())[:-2])

c:\Users\yaqoo\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\yaqoo\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 ReLU(inplace=True),
 MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False),
 Sequential(
   (0): Bottleneck(
     (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
     (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU(inplace=True)
     (downsample): Sequential(
       (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
       (1): BatchNorm2d(256, eps=1e-05, momentum

In [12]:
class Classification(nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet = nn.Sequential(*(list(models.resnet101(pretrained=True).children())[:-2]))
        self.Linear = nn.Linear(in_features=100, out_features=10)
    
    def forward(self, X):
        X =  self.resnet(X)
        X = X.view(X.shape[0], -1 )
        print(X.shape)
        X = self.Linear(X)
        return X
    

In [13]:
model_1= Classification().to(device)
model_1

Classification(
  (resnet): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2

In [14]:

optimizer = torch.optim.SGD(params=model_1.parameters(), lr=0.001)

In [15]:
class TrainingConfiguration:
    epochs: int=10
    learning_rate: float = 0.001
    loss_criteron :nn = nn.BCEWithLogitsLoss()


In [20]:
def train(model, train_loader, loss_criteron, optimizer):
    model.train()
    loss_sum = 0
    total_correct = 0 
    for batch, X, y in enumerate(train_loader):
        y_logits =  model(X.to(device))
        loss = loss_criteron(y_logits, y.to(device))
        y_pred = torch.argmax(torch.softmax(y_logits), dim=1)
        loss_sum += loss
        total_correct += torch.sum(torch.eq(y_pred, y)).item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    accuracy = total_correct/len(train_loader.dataset)
    avg_loss = loss_sum/len(train_loader.dataset)
    return accuracy, avg_loss


def val(model, test_loader, loss_criteron):
    model.eval()
    loss_sum = 0
    total_correct = 0 
    with torch.inference_mode():
        for X, y in test_loader:
            y_logits =  model(X.to(device))
            loss = loss_criteron(y_logits, y.to(device))
            y_pred = torch.argmax(torch.softmax(y_logits), dim=1)
            loss_sum += loss
            total_correct += torch.sum(torch.eq(y_pred, y)).item()
    
    accuracy = total_correct/len(test_loader.dataset)
    avg_loss = loss_sum/len(test_loader.dataset)
    return accuracy, avg_loss


def full_training(train_config: TrainingConfiguration = TrainingConfiguration(), model:nn.Module = model_1, optimizer=optimizer):
    train_loss = []
    train_acc = []
    test_loss = []
    test_acc = []
    print('Step 1: Preprocessing Image ')
    train, test = preprocess_image_folder_data()
    for epoch in tqdm.tqdm(range(train_config.epochs)):
        accuracy_train, loss_train = train(model, train , train_config.loss_criteron, optimizer)
        accuracy_test, loss_test = val(model, test, train_config.loss_criteron)
        train_loss.append(loss_train)
        train_acc.append(accuracy_train)
        test_loss.append(loss_test)
        test_acc.append(accuracy_test)
        print('Epoch:', epoch, '| train_acc:', accuracy_train, '| train_loss:', loss_train, ' | test_acc:', accuracy_test, '| test_loss:', loss_test )
    return model, train_acc, train_loss, test_acc, test_acc

        

        


In [ ]:
full_training()